In [1]:
import numpy as np

import random
import h5py
from keras import backend as K
from nn_util import *
from keras.models import *
from keras.layers import *
from keras.layers.core import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.initializers import *
from keras.models import load_model
from keras.losses import *
import scipy.io.wavfile as sciwav
import multiprocessing
from sklearn.cluster import KMeans, MiniBatchKMeans
from mem_top import mem_top

import os
import random
import time
import matplotlib
import matplotlib.pyplot as plt
import glob
import gc
import psutil

import operator
import math
import re

# for reproducibility
np.random.seed(1337) 
random.seed(1337)

# increase recursion limit for adaptive VQ
#import sys
#sys.setrecursionlimit(40000)

np.set_printoptions(formatter={'float_kind':'{:4f}'.format})

Using TensorFlow backend.


In [2]:
# control amount of GPU memory used
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [3]:
# external custom code I wrote
from load_data import *
from windowing import *
from pesq import *
from consts import *
from nn_blocks import *
from perceptual_loss import *
from evaluation import *

In [4]:
[train_paths, val_paths, test_paths], \
[train_waveforms, val_waveforms, test_waveforms], \
[train_procwave, val_procwave, test_procwave], \
[train_wparams, val_wparams, test_wparams], \
[train_windows, val_windows, test_windows] = load_data(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [5]:
# flatten all of the train windows into vectors
train_processed = np.array([i for z in train_windows for i in z])
train_processed = np.reshape(train_processed, (train_processed.shape[0], WINDOW_SIZE, 1))

# randomly shuffle data, if we want to
if (RANDOM_SHUFFLE):
    train_processed = np.random.permutation(train_processed)
    
print train_processed.shape
print np.mean(train_processed, axis=None)
print np.std(train_processed, axis=None)
print np.min(train_processed, axis = None)
print np.max(train_processed, axis = None)

(101814, 512, 1)
2.44142e-06
0.0994002
-1.0
1.0


In [6]:
CHANNEL_SIZE = WINDOW_SIZE / 2

# ---------------------------------------------------------------------------
# autoencoder: takes an audio window, compresses it, and tries to reconstruct it
# ---------------------------------------------------------------------------
def autoencoder_structure():   
    # - - - - - - - - - - - - - - - - - - - - -
    # parameters
    # - - - - - - - - - - - - - - - - - - - - -   
    NCHAN = 64
    FILT_SIZE = 9
    
    # feature extractor module, used in both encoder and decoder
    #     (structure is the same, but weights aren't shared)
    def feature_extractor():
        def f(inp):
            out = inp
            
            out = residual_block(NCHAN, FILT_SIZE, 1)(out)
            out = residual_block(NCHAN, FILT_SIZE, 2)(out)
            out = residual_block(NCHAN, FILT_SIZE, 4)(out)
            out = residual_block(NCHAN, FILT_SIZE, 8)(out)
            
            return out
        
        return f

    # - - - - - - - - - - - - - - - - - - - - -
    # encoder
    # - - - - - - - - - - - - - - - - - - - - -
    enc_input = Input(shape = (WINDOW_SIZE, 1))
    enc = enc_input
    
    # processing steps
    enc = channel_change_block(NCHAN, FILT_SIZE)(enc)  
    enc = feature_extractor()(enc)
    enc = downsample_block(NCHAN, FILT_SIZE)(enc)
    enc = feature_extractor()(enc)
    enc = channel_change_block(QUANT_CHANS, FILT_SIZE)(enc)
    
    # quantization (real numbers => soft bin assignments)
    pre_quant = Model(inputs = enc_input, outputs = enc)
    enc = SoftmaxQuantization()(enc)
    
    enc = Model(inputs = enc_input, outputs = enc, name = 'encoder')
    
    # - - - - - - - - - - - - - - - - - - - - -
    # decoder
    # - - - - - - - - - - - - - - - - - - - - -
    dec_input = Input(shape = (CHANNEL_SIZE, QUANT_CHANS, NBINS))
    dec = dec_input
    
    # "dequantization" (soft bin assignments => real numbers)
    dec = SoftmaxDequantization()(dec)
    post_dequant = Model(inputs = dec_input, outputs = dec)
    dec = Reshape((CHANNEL_SIZE, QUANT_CHANS))(dec)
    
    # processing steps
    dec = channel_change_block(NCHAN, FILT_SIZE)(dec)
    dec = feature_extractor()(dec)
    dec = upsample_block(NCHAN, FILT_SIZE)(dec)
    dec = feature_extractor()(dec)
    dec = channel_change_block(1, FILT_SIZE)(dec)
    
    dec = Model(inputs = dec_input, outputs = dec, name = 'decoder')
    
    # return both encoder and decoder
    return enc, dec, pre_quant, post_dequant

In [7]:
# map for load_model
KERAS_LOAD_MAP = {'PhaseShiftUp1D' : PhaseShiftUp1D,
                  'code_entropy' : code_entropy,
                  'code_sparsity' : code_sparsity,
                  'rmse' : rmse,
                  'SoftmaxQuantization' : SoftmaxQuantization,
                  'SoftmaxDequantization' : SoftmaxDequantization,
                  'DFT_REAL' : DFT_REAL,
                  'DFT_IMAG' : DFT_IMAG,
                  'MEL_FILTERBANKS' : MEL_FILTERBANKS,
                  'keras_dft_mag' : keras_dft_mag,
                  'keras_dct' : keras_dct,
                  'perceptual_transform' : perceptual_transform,
                  'perceptual_distance' : perceptual_distance}

In [8]:
# construct autoencoder
ac_input = Input(shape = (WINDOW_SIZE, 1))

encoder, decoder, pre_quant, post_dequant = autoencoder_structure()
ac_reconstructed = decoder(encoder(ac_input))
autoencoder = Model(inputs = [ac_input], outputs = [ac_reconstructed])

In [9]:
# model parameters
loss_weights = [30.0, 5.0, 10.0, 1.0]
loss_functions = [rmse, perceptual_distance, code_sparsity, code_entropy]
n_recons = 2
n_code = 2
assert(n_recons + n_code == len(loss_weights))
assert(len(loss_weights) == len(loss_functions))

In [10]:
# model specification
model_input = Input(shape = (WINDOW_SIZE, 1))
model_embedding = encoder(model_input)
model_pre_quant = pre_quant(model_input)
model_reconstructed = decoder(model_embedding)
model_post_dequant = post_dequant(model_embedding)

model = Model(inputs = [model_input], outputs = [model_reconstructed] * n_recons + \
                                            [model_embedding] * n_code)

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1511: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'decoder_1/add_22/add:0' shape=(?, 512, 1) dtype=float32>, <tf.Tensor 'decoder_1/add_22/add:0' shape=(?, 512, 1) dtype=float32>, <tf.Tensor 'encoder_1/softmax_quantization_1/cond/Merge:0' shape=(?, 256, 1, 32) dtype=float32>, <tf.Tensor 'encoder_1/softmax_quantization_1/cond/Merge:0' shape=(?, 256, 1, 32) dtype=float32>]
  ' Found: ' + str(self.outputs))


In [11]:
model.compile(loss = loss_functions,
              loss_weights = loss_weights,
              optimizer = Adam())

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 512, 1)            0         
_________________________________________________________________
encoder (Model)              (None, 256, 1, 32)        742447    
_________________________________________________________________
decoder (Model)              (None, 512, 1)            816399    
Total params: 1,558,846
Trainable params: 1,558,846
Non-trainable params: 0
_________________________________________________________________


In [12]:
# get untrained baseline for model
test_model_on_wav("./SA1.wav", "./train_output/SA1_uninit", autoencoder)

MSE:         154586.0
Avg err:     210.205
PESQ:        1.05026233196


[154586.23, 210.20505, 1.0502623319625854]

In [13]:
# saves current model
def save_model(prefix = 'best'):
    os.system('rm ./' + prefix + '_model.h5')
    os.system('rm ./' + prefix + '_auto.h5')
    #os.system('rm ./' + prefix + '_quant_bins.npy')
    
    model.save('./' + prefix + '_model.h5')
    autoencoder.save('./' + prefix + '_auto.h5')
    #np.save('./' + prefix + '_quant_bins.npy', K.eval(QUANT_BINS))
    
    f = h5py.File('./' + prefix + '_model.h5', 'r+')
    del f['optimizer_weights']
    f.close()

In [14]:
def evaluate_training(autoencoder, lead = ""):
    def set_evaluation(paths, eval_idxs):
        before_after_pairs = [run_model_on_wav(paths[i],
                                               autoencoder,
                                               argmax = True)
                              for i in eval_idxs]
        
        def thread_func(my_id, q):
            my_slice = np.arange(my_id, len(eval_idxs), NUM_THREADS)
            
            for i in my_slice:
                p = before_after_pairs[i]
                q.put(evaluation_metrics(p[0], p[1]))
        
        q = multiprocessing.Queue()
        threads = [multiprocessing.Process(target = thread_func,
                                           args = (i, q))
                   for i in xrange(0, NUM_THREADS)]
        [t.start() for t in threads]
        [t.join() for t in threads]
        results = np.array([q.get() for i in xrange(0, len(eval_idxs))])
        
        return results
    
    train_eval_idxs = random.sample(range(0, len(train_paths)), TRAIN_EVALUATE)
    val_eval_idxs = random.sample(range(0, len(val_paths)), VAL_EVALUATE)
    
    print lead + "Format: [MSE, avg err, PESQ]"
    
    # train set evaluation
    train_metrics = set_evaluation(train_paths,
                                   train_eval_idxs)
    print lead + "    Train: (mean)", np.mean(train_metrics, axis = 0)
    print lead + "    Train: (max) ", np.max(train_metrics, axis = 0)
    print lead + "    Train: (min) ", np.min(train_metrics, axis = 0)
    
    # validation set evaluation
    val_metrics = set_evaluation(val_paths,
                                 val_eval_idxs)
    print lead + "    Val:   (mean)", np.mean(val_metrics, axis = 0)
    print lead + "    Val:   (max) ", np.max(val_metrics, axis = 0)
    print lead + "    Val:   (min) ", np.min(val_metrics, axis = 0)
    
    # returns mean PESQ on validation
    return np.mean(val_metrics, axis = 0)[2]

In [15]:
X_train = np.copy(train_processed)
ntrain = X_train.shape[0]

BATCH_SIZE = 128
NUM_EPOCHS = 300
EPOCHS_BEFORE_QUANT_ON = 5

ORIG_BITRATE = 256.00
TARGET_BITRATE = 20.00
PRE_ENTROPY_RATE = ORIG_BITRATE * (float(CHANNEL_SIZE) / WINDOW_SIZE)

TARGET_ENTROPY = (TARGET_BITRATE / PRE_ENTROPY_RATE * 16.0)
TARGET_ENTROPY *= (STEP_SIZE / float(WINDOW_SIZE))
TARGET_ENTROPY_FUZZ = 0.1

TAU_CHANGE_RATE = 0.025
INITIAL_TAU = 0.5
MIN_TAU = 0.0

NUM_QUANT_VECS = 5000

STARTING_LR = 0.00025
ENDING_LR = 0.0001

print "Target entropy:", TARGET_ENTROPY

Target entropy: 2.34375


In [16]:
best_val_pesq = 0.0
K.set_value(tau, 0.0)
T_i = 0.0
K.set_value(QUANTIZATION_ON, False)

In [17]:
np.set_printoptions(formatter={'float_kind':'{:4f}'.format})
lead = "    "

for epoch in range(1, NUM_EPOCHS + 1):
    print "Epoch " + str(epoch) + ":"

    # present batches randomly each epoch
    lis = range(0, ntrain, BATCH_SIZE)
    random.shuffle(lis)
    num_batches = len(lis)
    
    # keep track of start time and current batch #
    i = 0
    startTime = time.time()
    for idx in lis:
        # cosine annealing for model's learning rate
        train_pct = T_i / float(NUM_EPOCHS)
        opt_lr = ENDING_LR + 0.5 * (STARTING_LR - ENDING_LR) * (1 + math.cos(3.14159 * train_pct))
        T_i += (1.0 / num_batches)
        K.set_value(model.optimizer.lr, opt_lr)
        
        batch = X_train[idx:idx+BATCH_SIZE, :,  :]
        nbatch = batch.shape[0]
               
        # train autoencoder
        a_y = [batch] * n_recons + \
              [np.zeros((nbatch, 1, 1, 1))] * n_code

        a_losses = model.train_on_batch(batch, a_y)
        
        # print statistics every 10 batches so we know what's going on
        if (i % 10 == 0):
            printStr = "        \r" + lead + str(i * BATCH_SIZE) + ": "
            print printStr,
            
            loss_arr = np.asarray(a_losses)
            print loss_arr,
            
            if (len(loss_weights) > 1 and len(loss_arr) > 1):
                for w in xrange(0, len(loss_weights)):
                    loss_arr[w + 1] *= loss_weights[w]
                print loss_arr,
            
            print K.get_value(tau), opt_lr,
        
        i += 1
    print ""
    
    # print elapsed time for epoch
    elapsed = time.time() - startTime
    print lead + "Total time for epoch: " + str(elapsed) + "s"
    
    # ---------------------------------------------------------
    # estimate code entropy from random samples (if quantization is on)
    # ---------------------------------------------------------
    if (K.get_value(QUANTIZATION_ON) > 0):
        NUM = 20000
        rows = np.random.randint(X_train.shape[0], size = NUM)
        to_predict = np.copy(X_train[rows, :])
        code = encoder.predict(to_predict, verbose = 0, batch_size = 128)
        
        all_onehots = np.reshape(code, (-1, QUANT_CHANS, NBINS))
        onehot_hist = np.sum(all_onehots, axis = 0)
        onehot_hist /= np.sum(onehot_hist, axis = 1, keepdims = True)
        
        # entropy for each channel
        channel_entropy = -np.sum(onehot_hist * np.log(onehot_hist + np.finfo(float).eps) / np.log(2.0),
                                  axis = 1)

        # total entropy
        entropy = np.sum(channel_entropy)

        print lead + "----------------"
        print lead + "Code entropy:", entropy

        # ---------------------------------------------------------
        # handle updating entropy weight (tau)
        # ---------------------------------------------------------
        old_tau = K.get_value(tau)

        if (entropy < TARGET_ENTROPY - TARGET_ENTROPY_FUZZ):
            new_tau = old_tau - TAU_CHANGE_RATE
            if (new_tau <= MIN_TAU):
                new_tau = MIN_TAU

            K.set_value(tau, new_tau)
            print lead + "Updated tau from", old_tau, "to", new_tau
        elif (entropy > TARGET_ENTROPY + TARGET_ENTROPY_FUZZ):
            new_tau = old_tau + TAU_CHANGE_RATE

            K.set_value(tau, new_tau)
            print lead + "Updated tau from", old_tau, "to", new_tau
        else:
            print lead + "Tau stays at", old_tau
    
    # ---------------------------------------------------------
    # evaluate autoencoder on training/validation data evey epoch
    # ---------------------------------------------------------
    startTime = time.time()
    print lead + "----------------"
    print lead + "Evaluating autoencoder..."
    
    
    metrics = test_model_on_wav("./SA1.wav", "./train_output/SA1_train_epoch" + str(epoch),
                                autoencoder, lead = lead, verbose = False, argmax = False)
    print lead + "SA1:         ", metrics
    if (K.get_value(QUANTIZATION_ON) > 0):
        metrics = test_model_on_wav("./SA1.wav", "./train_output/SA1_train_epoch" + str(epoch),
                                    autoencoder, lead = lead, verbose = False, argmax = True)
        print lead + "SA1 (arg):   ", metrics
    
    metrics = test_model_on_wav("./SX383.wav", "./train_output/SX383_train_epoch" + str(epoch),
                                autoencoder, lead = lead, verbose = False, argmax = False)
    print lead + "SX383:       ", metrics
    if (K.get_value(QUANTIZATION_ON) > 0):
        metrics = test_model_on_wav("./SX383.wav", "./train_output/SX383_train_epoch" + str(epoch),
                                    autoencoder, lead = lead, verbose = False, argmax = True)
        print lead + "SX383 (arg): ", metrics
    
    if (K.get_value(QUANTIZATION_ON) > 0):
        val_pesq = evaluate_training(autoencoder, lead)
        if (val_pesq > best_val_pesq and entropy <= TARGET_ENTROPY):
            print lead + "NEW best model! Validation mean-PESQ", val_pesq

            print lead + "Saving model..."
            save_model()
            best_val_pesq = val_pesq
            patience_epoch = epoch
        else:
            print lead + "Best validation mean-PESQ seen:", best_val_pesq
    
    elapsed = time.time() - startTime
    print lead + "Total time for evaluation: " + str(elapsed) + "s"
    
    gc.collect()
    process = psutil.Process(os.getpid())
    mem_used = process.memory_info().rss
    print lead + "Total memory usage: " + str(mem_used)
    
    # ---------------------------------------------------------
    # turn quantization on after a certain # of epochs
    # ---------------------------------------------------------
    if (epoch == EPOCHS_BEFORE_QUANT_ON):
        print lead + "----------------"
        print lead + "Turning quantization on!"
        
        random_windows = []
        for i in xrange(0, NUM_QUANT_VECS):
            w_idx = random.randint(0, train_processed.shape[0] - 1)
            random_windows.append(train_processed[w_idx])

        random_windows = np.array(random_windows)
        print lead + "    Selecting random code vectors for analysis..."
        encoded_windows = encoder.predict(random_windows, batch_size = 128, verbose = 0)

        print lead + "    K means clustering for bins initialization..."

        all_clustered = []
        cluster_scores = []
        for i in xrange(0, QUANT_CHANS):
            channel_values = encoded_windows[:, :, i, 0]
            channel_values = np.reshape(channel_values, (-1, 1))

            km = MiniBatchKMeans(n_clusters = NBINS).fit(channel_values)
            clustered = np.sort(km.cluster_centers_.flatten())
            all_clustered.append(clustered)

            cluster_score = np.sqrt(np.median(np.min(km.transform(channel_values), axis = 1)))
            cluster_scores.append(cluster_score)

        print lead + "    Done. Cluster scores:", cluster_scores
        
        clustered_bins = np.vstack(all_clustered)
        K.set_value(QUANTIZATION_ON, True)
        K.set_value(QUANT_BINS, clustered_bins)
        K.set_value(tau, INITIAL_TAU)

Epoch 1:
    101120:  [2.710840 0.005104 0.511543 0.000000 0.000000] [2.710840 0.153125 2.557715 0.000000 0.000000] 0.0 0.0002499959494691                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
    Total time for epoch: 148.846512079s
    ----------------
    Evaluating autoencoder...
    SA1:          [2481.2, 24.940832, 4.06865119934082]
    SX383:

KeyboardInterrupt: 

In [ ]:
if True:
    model = load_model('best_model.h5', KERAS_LOAD_MAP)
    autoencoder = load_model('best_auto.h5', KERAS_LOAD_MAP)
    encoder = autoencoder.layers[1]
    decoder = autoencoder.layers[2]
    K.set_value(QUANTIZATION_ON, True)

In [ ]:
enc = model.layers[1].layers
dec = model.layers[2].layers

In [ ]:
test_model_on_wav("./SA1.wav", "SA1_final", autoencoder)
test_model_on_wav("./SA1.wav", "SA1_final", autoencoder, argmax = True)

test_model_on_wav("./SX383.wav", "SX383_final", autoencoder)
test_model_on_wav("./SX383.wav", "SX383_final", autoencoder, argmax = True)

test_model_on_wav("./fiveYears.wav", "fy_final", autoencoder)
test_model_on_wav("./fiveYears.wav", "fy_final", autoencoder, argmax = True) 

In [ ]:
all_embed = encoder.predict(X_train[:10000], batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
probs = np.reshape(all_embed, (all_embed.shape[0] * all_embed.shape[1], QUANT_CHANS, NBINS))
hist = np.sum(probs, axis = 0)
hist /= np.sum(hist, axis = 1, keepdims = True)

for i in xrange(0, hist.shape[0]):
    print "--- CHANNEL", i, "---"
    
    sample_hist_bins = np.linspace(0, NBINS - 1, NBINS)
    plt.bar(sample_hist_bins, hist[i], align = 'center', width = 1)
    plt.show()
    
    entropy = 0
    for j in hist[i]:
        if (j < 1e-4): continue
        entropy += j * math.log(j, 2)
    entropy = -entropy
    print "Entropy of distribution:", entropy
    
    print "Hist:", hist[i]
    print "Bins:", K.eval(QUANT_BINS[i])
    print ""

In [ ]:
s = all_embed[0]

oh = K.eval(K.one_hot(K.argmax(K.variable(s)), NBINS))

print s.shape
print oh.shape

In [ ]:
[rate, data] = sciwav.read("./SA1.wav")
data = data.astype(np.float32)
processedWave, wparams = preprocess_waveform(data)
windows = extract_windows(processedWave, STEP_SIZE, OVERLAP_SIZE,
                          WINDOWING_MULT)

transformed = np.reshape(windows, (windows.shape[0], WINDOW_SIZE, 1))
embed = encoder.predict(transformed, batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
recons = decoder.predict(embed, batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
before_quantization = pre_quant.predict(transformed, batch_size = BATCH_SIZE, verbose = 1)
after_dequantization = post_dequant.predict(embed, batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
max_pct = np.max(embed[25], axis = -1)
print max_pct
print np.argmax(embed[25], axis = -1)
print np.sum(max_pct > 0.98) / float(max_pct.size)

In [ ]:
embed_max = np.max(embed, axis = -1)
print np.mean(embed_max)
print np.sum(embed_max > 0.98) / float(embed_max.size)

In [ ]:
idx = 44

orig = windows[idx].flatten()
recn = recons[idx].flatten()

print "Original"
plt.plot(orig)
ylim = plt.gca().get_ylim()
plt.show()

print "Reconstruction"
plt.plot(recn)
plt.ylim(ylim)
plt.show()

if (K.get_value(QUANTIZATION_ON) > 0):
    print "Before quantization"
    plt.plot(before_quantization[idx])
    ylim = plt.gca().get_ylim()
    plt.show()

    print "After dequantization"
    plt.plot(after_dequantization[idx])
    plt.ylim(ylim)
    plt.show()
else:
    print "Embedding"
    plt.plot(before_quantization[idx])
    ylim = plt.gca().get_ylim()
    plt.show()

print "Error"
plt.plot(abs(orig - recn))
plt.show()